In [1]:
import torch

使用PyTorch计算梯度数值
PyTorch的Autograd模块实现了深度学习的算法中的向传播求导数，在张量（Tensor类）上的所有操作，Autograd都能为他们自动提供微分，简化了手动计算导数的复杂过程。

在0.4以前的版本中，Pytorch 使用 Variable 类来自动计算所有的梯度。Variable类主要包含三个属性： data：保存Variable所包含的Tensor；grad：保存data对应的梯度，grad也是个Variable，而不是Tensor，它和data的形状一样；grad_fn：指向一个Function对象，这个Function用来反向传播计算输入的梯度。

从0.4起， Variable 正式合并入Tensor类，通过Variable嵌套实现的自动微分功能已经整合进入了Tensor类中。虽然为了代码的兼容性还是可以使用Variable(tensor)这种方式进行嵌套，但是这个操作其实什么都没做。

所以，以后的代码建议直接使用Tensor类进行操作，因为官方文档中已经将Variable设置成过期模块。

要想通过Tensor类本身就支持了使用autograd功能，只需要设置.requires_grad=True

Variable类中的的grad和grad_fn属性已经整合进入了Tensor类中

Autograd
在张量创建时，通过设置 requires_grad 标识为Ture来告诉Pytorch需要对该张量进行自动求导，PyTorch会记录该张量的每一步操作历史并自动计算

In [2]:
x = torch.rand(5,5,requires_grad=True)
x

tensor([[0.6562, 0.6356, 0.8144, 0.2150, 0.1576],
        [0.4427, 0.8118, 0.1933, 0.8791, 0.2751],
        [0.6696, 0.3054, 0.1316, 0.6714, 0.1696],
        [0.5216, 0.8208, 0.6589, 0.7055, 0.8811],
        [0.1635, 0.3448, 0.3132, 0.5903, 0.2387]], requires_grad=True)

In [3]:
y = torch.rand(5,5,requires_grad=True)
y


tensor([[0.2104, 0.2640, 0.1086, 0.5894, 0.2388],
        [0.1272, 0.5330, 0.2432, 0.7399, 0.8034],
        [0.6726, 0.3061, 0.8336, 0.0846, 0.3252],
        [0.3366, 0.8975, 0.3879, 0.2534, 0.5792],
        [0.3557, 0.3223, 0.2286, 0.9494, 0.9107]], requires_grad=True)

PyTorch会自动追踪和记录对与张量的所有操作，当计算完成后调用.backward()方法自动计算梯度并且将计算结果保存到grad属性中。

In [5]:
z = torch.sum(x+y)
z

tensor(23.5680, grad_fn=<SumBackward0>)

在张量进行操作后，grad_fn已经被赋予了一个新的函数，这个函数引用了一个创建了这个Tensor类的Function对象。 Tensor和Function互相连接生成了一个非循环图，它记录并且编码了完整的计算历史。每个张量都有一个.grad_fn属性，如果这个张量是用户手动创建的那么这个张量的grad_fn是None。

下面我们来调用反向传播函数，计算其梯度

简单的自动求导

In [6]:
z.backward()
print(x.grad,y.grad)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]) tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])


In [7]:
z.shape

torch.Size([])

如果Tensor类表示的是一个标量（即它包含一个元素的张量），则不需要为backward()指定任何参数，但是如果它有更多的元素，则需要指定一个gradient参数，它是形状匹配的张量。 以上的 z.backward()相当于是z.backward(torch.tensor(1.))的简写。 这种参数常出现在图像分类中的单标签分类，输出一个标量代表图像的标签。

复杂的自动求导

In [9]:
x = torch.rand(5,5,requires_grad=True)
y = torch.rand(5,5,requires_grad=True)
z = x**2+y**3
z

tensor([[0.2196, 0.6641, 0.8271, 0.2469, 1.0417],
        [1.1242, 0.4062, 0.0240, 1.4470, 0.3257],
        [0.1191, 1.4285, 0.4571, 0.8812, 0.6360],
        [1.0351, 0.2743, 0.2984, 0.2028, 0.0476],
        [0.9826, 1.0011, 1.0959, 0.0193, 1.0815]], grad_fn=<AddBackward0>)

In [108]:
#我们的返回值不是一个标量，所以需要输入一个大小相同的张量作为参数，这里我们用ones_like函数根据x生成一个张量
z.backward(torch.ones_like(x))
print(x.grad)

tensor([[0.7335, 1.4131, 1.7910, 0.4069, 0.9169],
        [1.6470, 1.2746, 0.1936, 1.7362, 1.1199],
        [0.6827, 1.3164, 0.7476, 1.2393, 1.4904],
        [1.0018, 0.5926, 1.0118, 0.1420, 0.1726],
        [1.6347, 1.9785, 1.2138, 0.2604, 0.8696]])


我们可以使用with torch.no_grad()上下文管理器临时禁止对已设置requires_grad=True的张量进行自动求导。这个方法在测试集计算准确率的时候会经常用到，例如：

In [109]:
with torch.no_grad():
    print((x +y*2).requires_grad)

False


使用.no_grad()进行嵌套后，代码不会跟踪历史记录，也就是说保存的这部分记录会减少内存的使用量并且会加快少许的运算速度。

backward()参数解析，https://zhuanlan.zhihu.com/p/29923090  https://www.cnblogs.com/zhouyang209117/p/11023160.html

In [129]:
a  = torch.FloatTensor([[2,3],[1,2]])

w = torch.ones((2,3),requires_grad=True)




In [130]:
a

tensor([[2., 3.],
        [1., 2.]])

In [131]:
w

tensor([[1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)

In [132]:
out = torch.mm(a,w)

grad = torch.tensor([[1,1,1],[1,1,2]])

In [133]:
grad

tensor([[1, 1, 1],
        [1, 1, 2]])

In [134]:
out.backward(grad)

In [135]:

w.grad

tensor([[3., 3., 4.],
        [5., 5., 7.]])

In [94]:
w.grad.shape


torch.Size([2, 3])

In [95]:
out

tensor([[5., 5., 5.],
        [3., 3., 3.]], grad_fn=<MmBackward0>)

Autograd 过程解析
为了说明Pytorch的自动求导原理，我们来尝试分析一下PyTorch的源代码，虽然Pytorch的 Tensor和 TensorBase都是使用CPP来实现的，但是可以使用一些Python的一些方法查看这些对象在Python的属性和状态。 Python的 dir() 返回参数的属性、方法列表。z是一个Tensor变量，看看里面有哪些成员变量。

In [110]:
dir(z)

['H',
 'T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed_

返回很多，我们直接排除掉一些Python中特殊方法（以__开头和结束的）和私有方法（以_开头的，直接看几个比较主要的属性： .is_leaf：记录是否是叶子节点。通过这个属性来确定这个变量的类型 在官方文档中所说的“graph leaves”，“leaf variables”，都是指像x，y这样的手动创建的、而非运算得到的变量，这些变量称为创建变量。 像z这样的，是通过计算后得到的结果称为结果变量。

一个变量是创建变量还是结果变量是通过.is_leaf来获取的。

In [111]:
print("x.is_leaf="+str(x.is_leaf))
print("z.is_leaf="+str(z.is_leaf))

x.is_leaf=True
z.is_leaf=False


x是手动创建的没有通过计算，所以他被认为是一个叶子节点也就是一个创建变量，而z是通过x与y的一系列计算得到的，所以不是叶子结点也就是结果变量。

为什么我们执行z.backward()方法会更新x.grad和y.grad呢？ .grad_fn属性记录的就是这部分的操作，虽然.backward()方法也是CPP实现的，但是可以通过Python来进行简单的探索。

grad_fn：记录并且编码了完整的计算历史

In [112]:
z.grad_fn

grad_fn是一个AddBackward0类型的变量 AddBackward0这个类也是用Cpp来写的，但是我们从名字里就能够大概知道，他是加法(ADD)的反反向传播（Backward），看看里面有些什么东西

In [113]:
dir(z.grad_fn)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_register_hook_dict',
 '_saved_alpha',
 'metadata',
 'name',
 'next_functions',
 'register_hook',
 'register_prehook',
 'requires_grad']

next_functions就是grad_fn的精华

In [114]:
z.grad_fn.next_functions

((<PowBackward0 at 0x24e946105b0>, 0), (<PowBackward0 at 0x24e94613400>, 0))

next_functions是一个tuple of tuple of PowBackward0 and int。

为什么是2个tuple ？ 因为我们的操作是z= x**2+y**3 刚才的AddBackward0是相加，而前面的操作是乘方 PowBackward0。tuple第一个元素就是x相关的操作记录

In [115]:
xg = z.grad_fn.next_functions[0][0]
dir(xg)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_raw_saved_self',
 '_register_hook_dict',
 '_saved_exponent',
 '_saved_self',
 'metadata',
 'name',
 'next_functions',
 'register_hook',
 'register_prehook',
 'requires_grad']

In [116]:
x_leaf=xg.next_functions[0][0]
type(x_leaf)

AccumulateGrad

在PyTorch的反向图计算中，AccumulateGrad类型代表的就是叶子节点类型，也就是计算图终止节点。AccumulateGrad类中有一个.variable属性指向叶子节点。

In [117]:
x_leaf.variable

tensor([[0.3668, 0.7066, 0.8955, 0.2035, 0.4584],
        [0.8235, 0.6373, 0.0968, 0.8681, 0.5600],
        [0.3414, 0.6582, 0.3738, 0.6196, 0.7452],
        [0.5009, 0.2963, 0.5059, 0.0710, 0.0863],
        [0.8174, 0.9893, 0.6069, 0.1302, 0.4348]], requires_grad=True)

这个.variable的属性就是我们的生成的变量x

In [118]:
print("x_leaf.variable的id:"+str(id(x_leaf.variable)))
print("x的id:"+str(id(x)))

x_leaf.variable的id:2536372792960
x的id:2536372792960


In [119]:
assert(id(x_leaf.variable)==id(x))

这样整个规程就很清晰了：

当我们执行z.backward()的时候。这个操作将调用z里面的grad_fn这个属性，执行求导的操作。
这个操作将遍历grad_fn的next_functions，然后分别取出里面的Function（AccumulateGrad），执行求导操作。这部分是一个递归的过程直到最后类型为叶子节点。
计算出结果以后，将结果保存到他们对应的variable 这个变量所引用的对象（x和y）的 grad这个属性里面。
求导结束。所有的叶节点的grad变量都得到了相应的更新
最终当我们执行完c.backward()之后，a和b里面的grad值就得到了更新。

扩展Autograd
如果需要自定义autograd扩展新的功能，就需要扩展Function类。因为Function使用autograd来计算结果和梯度，并对操作历史进行编码。 在Function类中最主要的方法就是forward()和backward()他们分别代表了前向传播和反向传播。

一个自定义的Function需要一下三个方法：

__init__ (optional)：如果这个操作需要额外的参数则需要定义这个Function的构造函数，不需要的话可以忽略。

forward()：执行前向传播的计算代码

backward()：反向传播时梯度计算的代码。 参数的个数和forward返回值的个数一样，每个参数代表传回到此操作的梯度。

In [120]:
# 引入Function便于扩展
from torch.autograd.function import Function

In [121]:
# 定义一个乘以常数的操作(输入参数是张量)
# 方法必须是静态方法，所以要加上@staticmethod
class MulConstant(Function):
    @staticmethod
    def forward(ctx, tensor, constant):
        # ctx 用来保存信息这里类似self，并且ctx的属性可以在backward中调用
        ctx.constant=constant
        return tensor *constant
    @staticmethod
    def backward(ctx, grad_output):
        # 返回的参数要与输入的参数一样.
        # 第一个输入为3x3的张量，第二个为一个常数
        # 常数的梯度必须是 None.
        return grad_output, None

定义完我们的新操作后，我们来进行测试

In [122]:
a=torch.rand(3,3,requires_grad=True)
b=MulConstant.apply(a,5)
print("a:"+str(a))
print("b:"+str(b)) # b为a的元素乘以5

a:tensor([[0.1595, 0.5293, 0.6449],
        [0.9744, 0.5376, 0.6854],
        [0.3543, 0.4170, 0.1068]], requires_grad=True)
b:tensor([[0.7974, 2.6464, 3.2245],
        [4.8720, 2.6878, 3.4269],
        [1.7714, 2.0852, 0.5338]], grad_fn=<MulConstantBackward>)


反向传播，返回值不是标量，所以backward方法需要参数

In [123]:
b.backward(torch.ones_like(a))
a.grad

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])